Prepares the dataset for classification by:
- Accessing Parquet data from an AWS S3 bucket
- Formatting and standardizing training data
- Addressing class imbalance using synthetic data generation techniques
- Preprocessing test data for evaluation and cross-validation

In [10]:
# "standard"
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

# misc
import pickle, os
from IPython.display import clear_output
from imblearn.over_sampling import SMOTE
from collections import Counter

# Preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# src
import sys
from pathlib import Path
script_dir = Path(os.getcwd()).parent
src_dir = script_dir / 'src'
sys.path.append(str(src_dir))
from img_preprocessing import dict_to_image

clear_output(wait=False)


Create results directory

In [31]:
script_dir = Path(os.getcwd()).parent
results_dir = script_dir / 'results' / 'notebook_00'
os.makedirs(results_dir, exist_ok=True)

Load and format training data

In [ ]:
'''
Label meanings
0 - Mild dementia
1 - Moderate dementia
2 - No dementia
3 - Very mild dementia
'''
Lab = ['Mild', 'Moderate', 'None', 'Very Mild']

# Define the path to the raw data folder
data_dir = Path(os.getcwd()).parent / 'data' / 'raw' 
parquet_file = os.path.join(data_dir, "train-00000-of-00001-c08a401c53fe5312.parquet")

# Load the data
train = pd.read_parquet(parquet_file)

# Display data shape or a preview
train.head()

Convert data to readable format

In [ ]:
train['img_arr'] = train['image'].apply(dict_to_image)
train.drop("image", axis=1, inplace=True)
train.head()

Load and convert test data

In [ ]:
parquet_file = os.path.join(data_dir, "test-00000-of-00001-44110b9df98c5585.parquet")
test = pd.read_parquet(parquet_file)
test.head() 

# Also convert to readable format
test['img_arr'] = test['image'].apply(dict_to_image)
test.drop("image", axis=1, inplace=True)
test.head()

Examine some sample images to check data quality

In [ ]:
train_lab_idx = np.asarray(train.iloc[:].label)

f, ax = plt.subplots(4, 4, figsize=(5, 5))
for lab in range(4):
    for ex in range(4):
    
        class_lab = np.argwhere(train_lab_idx == 1)
        current_idx = np.random.randint(len(class_lab)-1,size = 1)
        current_idx = np.asarray(current_idx)
        
        ax[ex, lab].axis('off')
        ax[ex, lab].imshow(train.iloc[class_lab[current_idx[0]][0]].img_arr, cmap = "gray")
        if ex == 0: ax[ex, lab].set_title(Lab[lab])

Clearly, images show different slices within the brain, which may be a major confound...

Explore distribution of dataset classes

In [ ]:
colors = ['#aec7e8', '#ffbb78', '#98df8a', '#ff9896']

f, ax = plt.subplots(1, 2, figsize=(10, 3))
unique, counts = np.unique(np.asarray(train.iloc[:].label), return_counts=True)
ax[0].bar(unique, counts, color=colors)
ax[0].set_xticks(unique)
ax[0].set_xticklabels(Lab, rotation=45)
ax[0].set_title('Training')
ax[0].set_xlabel('Class')
ax[0].set_ylabel('# of images')

unique, counts = np.unique(np.asarray(test.iloc[:].label), return_counts=True)
ax[1].bar(unique, counts, color=colors)
ax[1].set_xticks(unique)
ax[1].set_xticklabels(Lab, rotation=45)
ax[1].set_title('Testing')
ax[1].set_xlabel('Class')
ax[1].set_ylabel('# of images')

We can see that there is an obvious imbalance across classes in both the training and testing sets. However, each class has been proportionally split between the two. Nevertheless, let's attempt to balance the training set such that the model sees equal numbers of each class. To avoid overfitting (e.g. simple resampling), we use the SMOTE method here to synthetically generate new data based on what is available.

In [ ]:
# Extract features and labels
train_tmp = np.array([img.flatten() for img in train['img_arr']])
train_lab_tmp = train['label']

# Apply SMOTE
smote = SMOTE(random_state=42)
train_smote, train_smote_lab = smote.fit_resample(train_tmp.reshape(-1, 128*128), train_lab_tmp)
train_smote = train_smote.reshape(-1, 128, 128)

# Create a new DataFrame with the resampled data
train_smote = pd.DataFrame({'label': train_smote_lab, 'img_arr': [img.tolist() for img in train_smote]})
train_smote_lab = train_smote['label']

# Plot the distribution of the different classes
colors = ['#aec7e8', '#ffbb78', '#98df8a', '#ff9896']

f, ax = plt.subplots(1, 1, figsize=(5, 3))
unique, counts = np.unique(train_smote_lab, return_counts=True)
ax.bar(unique, counts, color=colors)
ax.set_xticks(unique)
ax.set_xticklabels(Lab, rotation=45)
ax.set_title('Resampled Training')
ax.set_xlabel('Class')
ax.set_ylabel('# of images')

Now we see that the training set is balanced across classes. Let's inspect some of the new data for quality.

In [ ]:
first_indices = {}
for num in range(0, 4):
    first_index = next((i for i, x in enumerate(train_smote_lab[5121:], start=5121) if x == num), None)
    first_indices[num] = first_index

for label, index in first_indices.items():
    if index is not None:
        num_synthetic_images = len(train_smote_lab) - index
        print(f"Class {Lab[label]}: {num_synthetic_images} synthetic images")

# Visualize the images from the first_indices values
f, ax = plt.subplots(1, 4, figsize=(20, 5))
for i, label in enumerate(np.unique(train_smote_lab)):
    if first_indices[label] is not None:
        first_image = np.array(train_smote.iloc[first_indices[label]]['img_arr']).reshape(128, 128)
        ax[i].imshow(first_image, cmap='gray')
        ax[i].set_title(f"{Lab[label]}: SMOTE")
    ax[i].axis('off')

In these cases, SMOTE did not successfully generate accurate synthetic data. While some realistic features are apparent, the overall quality is poor. Therefore, we will attemp to balance classes using a classic augmentation approach.

In [23]:
train_tmp = np.array([img for img in train['img_arr']])
train_tmp = train_tmp.reshape(-1, 128, 128, 1)

train_lab_tmp = train['label'].values

# Calculate class distribution
class_counts = Counter(train_lab_tmp)

# Split the data by class
class_images = {}  # Dictionary to hold images by class
class_labels = {}  # Dictionary to hold labels by class

for img, label in zip(train_tmp, train_lab_tmp):
    if label not in class_images:
        class_images[label] = []
        class_labels[label] = []
    class_images[label].append(img)
    class_labels[label].append(label)

# Plot one image from class_images
label_to_plot = list(class_images.keys())[0]  # Select the first label
image_to_plot = class_images[label_to_plot][0]  # Select the first image of that label

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Augment the minority classes
max_samples = max(len(images) for images in class_images.values())

train_balanced = []  # List to hold all augmented images
train_lab_balanced = []  # List to hold all augmented labels

for label, images in class_images.items():
    num_augmentations = max_samples - len(images)  # How many more samples we need

    # Minority class
    if num_augmentations > 0:
        for img_batch, lab_batch in datagen.flow(np.array(images), np.array([label] * len(images)), batch_size=32):
            train_balanced.extend(img_batch)
            train_lab_balanced.extend(lab_batch)
            num_augmentations -= len(img_batch)
            if num_augmentations <= 0:
                break
    
    # Add the original images of the class to the balanced set
    train_balanced.extend(images)
    train_lab_balanced.extend([label] * len(images))


train_balanced = np.array(train_balanced)
train_lab_balanced = np.array(train_lab_balanced)

Plot examples of original and augmented/generated data

In [ ]:
f, ax = plt.subplots(1, 4, figsize=(20, 5))

# Plot balanced images in order of mild, moderate, non, very mild
order = ['Mild', 'Moderate', 'None', 'Very Mild']
label_order = [Lab.index(o) for o in order]

for i, label in enumerate(label_order):
    if Lab[label] != 'None':
        balanced_images = [img for img, lab in zip(train_balanced, train_lab_balanced) if lab == label]
        image_to_plot = balanced_images[0]  # Select the first balanced image of that label
        ax[i].imshow(image_to_plot.reshape(128, 128), cmap='gray')
        ax[i].set_title(f"{Lab[label]}: Balanced", fontsize=14) 
    ax[i].axis('off')

plt.show()

We have to assume that the original images were all registerer to a standard space. While the augmentation created new instances of the minority classes, these images are now not in the same anatomical space. Therefore, this introduces a new bias in that important features, such as specific brain structures, cannot be interpreted in the context of classification. This is in contrast to SMOTE - where synethic data was created in the same anatomical space as the original data, the quality was poor and would also lead to feature bias. Therefore, while less than ideal, using the original unbalanced dataset may be the best option in this case. We can still determine if the class inbalance meaningfully impacts the overall or class-specific classification.

Save the preprocessed dataset for model testing

In [ ]:
# Organize preprocessed data
data_preproc = {
    'train_data': np.array([img for img in train['img_arr']]),
    'train_labels': train['label'].values,
    'test_data': np.array([img for img in test['img_arr']]),
    'test_labels': test['label'].values
}

file_name = results_dir / 'data_preprocessed.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(data_preproc, f)

print(f"Data saved to {file_name}")

For models that require X-fold validation, we save a separate dataset

In [ ]:
# Combine training and test sets together
data = np.concatenate((data_preproc['train_data'], data_preproc['test_data']), axis=0)
labels = np.array(np.concatenate((data_preproc['train_labels'], data_preproc['test_labels']), axis=0))

# Split data into 5 train/test sets
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_idx_fold = []
test_idx_fold = []
for train_idx, test_idx in skf.split(labels, labels):
    y_train, y_test = labels[train_idx], labels[test_idx]
    train_idx_fold.append(train_idx)
    test_idx_fold.append(test_idx)
    
    # Check class balance across train/test sets
    print(f"Training class distribution: {Counter(y_train)}")
    print(f"Testing class distribution: {Counter(y_test)}")
    print()

# Organize preprocessed data
data_preproc_xfold = {
    'data': data,
    'labels': labels,
    'train_idx_fold': train_idx_fold,
    'test_idx_fold': test_idx_fold
}

file_name = results_dir / 'data_preprocessed_xfold.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(data_preproc_xfold, f)

print(f"Data saved to {file_name}")